# Лабораторная работа 3. 
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы Самат уулу Максат БВТ2101
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом. 

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса. 

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена

### Выполнение:

In [8]:
import math
import copy
import time

import networkx as nx
import pandas as pd

from pyvis.network import Network

In [9]:
def graph_list(m: list) -> list:
    granny = []
    for i in range(len(m)):
        for j in range(len(m)):
            if m[i][j] != math.inf and m[i][j] != 0:
                granny.append([i, j, m[i][j]])

    return granny

def list_graph(e: list, n: int) -> list:
    granny = [[math.inf for x in range(n)] for x in range(n)]

    for x in range(n):
        granny[x][x] = 0
    
    for edge in e:
        granny[edge[0]][edge[1]] = edge[-1]
    
    return granny

#### Визуализация:

In [10]:
def vis(m, way, title):
    G = nx.Graph()
    e = graph_list(m)

    for i in range(len(m)):
        if i in way:
            G.add_node(i, label=f'{i}', color="red")
        else:
            G.add_node(i, label=f'{i}', color="grey")


    for edge in e:
        G.add_edge(edge[0], edge[1], title=edge[-1], color="grey")

    for i in range(len(way)-1, 0, -1):
        G.add_edge(way[i], way[i-1], title=edge[-1], color="red")
        

    nt = Network(notebook=True, directed=True)
    nt.from_nx(G)
    nt.show(f'{title}.html')

#### Чтение данных из файла:
***

##### Чтение матрицы смежности из файла:

In [11]:
def read_adjancy(title: str):
    matrix = []

    with open(title) as f:
        for line in f:
            if line != '\n':
                l = line.strip().split("]")
                l[0] = l[0][1:]
                l = l[0].split(", ")
                matrix.append([int(x) if x.isdigit() and x != '0' else math.inf for x in l])
            else: break

        for i in range(len(matrix)):
            matrix[i][i] = 0

    return matrix
    
for i in read_adjancy("adjacency.txt"):
    print(i)

[0, 2, inf, 3, 1, inf, inf, 10]
[2, 0, 4, inf, inf, inf, inf, inf]
[inf, 4, 0, inf, inf, inf, inf, 3]
[3, inf, inf, 0, inf, inf, inf, 8]
[1, inf, inf, inf, 0, 2, inf, inf]
[inf, inf, inf, inf, 2, 0, 3, inf]
[inf, inf, inf, inf, inf, 3, 0, 1]
[10, inf, 3, 8, inf, inf, 1, 0]


##### Чтение матрицы инцидентности из файла:

In [12]:
def read_incidence(title: str):
    m, e = [], []

    with open(title) as f:
        for line in f:
            if line != '\n':
                l = line.strip().split("]")
                l[0] = l[0][1:]
                l = l[0].split(", ")
                m.append([int(x) for x in l])
            else: break

    for i in range(len(m[-1])):
        for j in range(len(m)):
            for k in range(len(m)):
                if j != k and m[j][i] != 0:
                    if m[j][i] == m[k][i]:
                        e.append([j, k, m[j][i]])
                    elif m[j][i] == -1*m[k][i]:
                        if m[j][i] < m[k][i]:
                            e.append([k, j, m[k][i]])
                        else:
                            e.append([j, k, m[j][i]])

    return list_graph(e, len(m))

for i in read_incidence("incidence.txt"):
    print(i)

[0, 2, inf, 3, 1, inf, inf, 10]
[2, 0, 4, inf, inf, inf, inf, inf]
[inf, 4, 0, inf, inf, inf, inf, 3]
[3, inf, inf, 0, inf, inf, inf, 8]
[1, inf, inf, inf, 0, 2, inf, inf]
[inf, inf, inf, inf, 2, 0, 3, inf]
[inf, inf, inf, inf, inf, 3, 0, 1]
[10, inf, 3, 8, inf, inf, 1, 0]


##### Чтение нескольких матриц смежности из файла:

In [13]:
def read_adjancies(title: str):
    matrixes = []

    with open(title) as f:
        matrixes = f.read().split("]\n")

        for i in range(len(matrixes)):
            matrixes[i] += ']'

            if matrixes[i][0] == '\n': 
                matrixes[i] = matrixes[i][1:]

            with open("temp.txt", 'w') as lamborghini:
                lamborghini.write(matrixes[i])

            matrixes[i] = read_adjancy("temp.txt")
    
    return matrixes

#### Входные данные: 

In [14]:
start = 0
end = 7

matrix = read_adjancy("adjacency.txt")

#### Алгоритмы:
***

##### Алгоритм Флойда-Уоршелла

In [15]:
def floyd_warshall(m, start, end):

    # Количество вершин
    n = len(m)
    # Матрица предыдущих вершин
    p = [[None for x in range(n)] for y in range(n)]


    # Функция возвращает кратчайший путь.
    def path(p, v, u):
        result = [u]
        while u != None:
            u = p[u][v]
            result.append(u)

        result[-1] = v
        return result
    

    for k in range(n):
        for i in range(n):
            for j in range(n):
                # Проход через дополнительную вершин
                d = m[i][k] + m[k][j]
                
                # Если проход через дополнительную вершину короче
                # прохода без неё, то обновляем вес ребра.
                if m[i][j] > d:
                    m[i][j] = d
                    p[i][j] = k

    return [m, path(p, start, end)]


ma = copy.deepcopy(matrix)
result = floyd_warshall(ma, start, end)
print("length =", result[0][start][end], "\npath =", result[1])
vis(matrix, result[1], "floyd_warshall")

length = 7 
path = [7, 6, 5, 4, 0]
floyd_warshall.html


##### Алгоритм Дейкстры

In [16]:
def dijkstra(m, start, end):
    
    # Количество вершин
    n = len(m)
    # Количество вершин
    v = start
    # Массив кратчайших путей до n-ой вершины
    d = [math.inf]*n
    # Массив предыдущих вершин
    prev = [None]*n
    # Вершины, доступные для поиска кратчайшего пути
    q = [x for x in range(n)]
    
    # Обнуляем кратчайший путь из стартовой вершины в себя.
    d[v] = 0

    # Функция возвращает вершину, путь до которой минимален.
    def v_min(t, q):
        # Вершина, путь до которой минимален
        vmin = -1
        # Кратчайший путь до этой вершины
        a = math.inf

        # Ищем кратчайший путь до непосещённой вершины.
        for i, j in enumerate(t):
            if j < a and i in q:
                a = j
                vmin = i

        return vmin
  

    # Пока существуют непосещённые вершины считаем 
    # кратчайшие пути до них с учётом посёщенных вершин. 
    while v != -1:
        for i, dw in enumerate(m[v]):
            if i in q:
                f = d[v] + dw
                if f < d[i]:
                    d[i] = f
                    prev[i] = v

        v = v_min(d, q)

        # Удаяляем вершину из массива доступных вершин.
        if v >= 0:
            q.remove(v)

    # Поиск кратчайшего пути
    route = [end]
    while end != start and route[-1] != None:
        end = prev[route[-1]]
        route.append(end)
    return [d, route]


result = dijkstra(matrix, start, end)
print("length =", result[0][end], "\npath =", result[1])
vis(matrix, result[1], "dijkstra")

length = 7 
path = [7, 6, 5, 4, 0]
dijkstra.html


##### Алгоритм Беллмана-Форда

In [17]:
def bellman_ford(m, start, end):

    # Количество вершин
    n = len(m)
    # Массив ребёр
    e = graph_list(m)
    # Матрица предыдущих вершин
    p = [[None for x in range(n-1)] for x in range(n)]
    # Матрица кратчайших путей
    a = [[math.inf for x in range(n-1)] for x in range(n)]

    # Обнуляем кратчайший путь из стартовой вершины в себя.
    a[start][0] = 0

    for i in range(1, n-1):
        for edge in e:
            # Проход через вершину
            d = a[edge[0]][i-1] + edge[-1]

            # Если проход через вершину короче 
            # прохода из матрицы кратчайших путей, 
            # то заменяем его.
            if d < a[edge[1]][i]:
                a[edge[1]][i] = d
                p[edge[1]][i] = edge[0]
    
    # Количество ребёр в кратчайшем пути
    j = a[end].index(min(a[end]))
    # Вершина, которую необходимо достичь
    i = end                                                 
    # Кратчайший путь
    route = [math.inf]*(j+1)         
    # Начало кратчайшего пути
    route[0] = start                                        

    # Поиск кратчайшего пути
    while j > 0:
        route[j] = i
        i = p[i][j]
        j -= 1
    
    return [a, route]


result = bellman_ford(matrix, start, end)
print("length =", result[0][end][len(result[1])-1], "\npath =", result[1][::-1])
vis(matrix, result[1][::-1], "bellman_ford")

length = 7 
path = [7, 6, 5, 4, 0]
bellman_ford.html


##### Алгоритм Джонсона

In [18]:
def johnson(m, start, end):
    
    # Массив ребёр
    e = graph_list(m)
    # Количество ребёр
    e_num = len(e)
    # Количество вершин
    v_num = len(m)
    # Матрица кратчайших путей
    d = [[math.inf for x in range(v_num)] for x in range(v_num)]
    # Кратчайший путь
    p = []
    
    # Добавляем временную вершину, из которой 
    # исходят рёбра с нулевым весом.
    for i in range(v_num):
        e.append([v_num, i, 0])

    
    # Функция возвращает массив кратчайчих путей до вершин. 
    # Является упрощенным алгоритмом Беллмана-Форда.
    def easy_bellman_ford(e, n):
        d = [math.inf]*n
        d[n-1] = 0
        for i in range(1, n-1):
            for edge in e:
                if d[edge[1]] > d[edge[0]] + edge[-1]:
                    d[edge[1]] = d[edge[0]] + edge[-1]
    
        return d
    
    # Массив кратчайших путей от временной
    # вершины до всех остальных.
    h = easy_bellman_ford(e, v_num+1)

    # Изменяем веса ребёр так,
    # чтобы сделать их положительными.
    for i in range(e_num):
        e[i][-1] += h[e[i][0]] - h[e[i][1]]
    
    # Удаляем временную вершину.
    del e[-v_num:-1]
    e.pop()

    # Используем алгоритм Дейкстры для нахождения
    # кратчайших путей до заданной вершины.
    m = list_graph(e, v_num)
    for i in range(v_num):
        temp_result = dijkstra(m, i, end)
        p.append(temp_result[-1])

        # Возвращаем исходные веса ребёр.
        for j in range(v_num):
            d[i][j] = temp_result[0][j] + h[j] - h[i]
    

    return [d, p]

result = johnson(matrix, start, end)
print("length =", result[0][start][end], "\npath =", result[1][start])
vis(matrix, result[1][start], "johnson")

length = 7 
path = [7, 6, 5, 4, 0]
johnson.html


##### Алгоритм Левита

In [19]:
def levit(m, start, end):
    
    # Количество вершин
    n = len(m)
    # Массив кратчайших путей до n-ой вершины
    d = [math.inf]*n
    # Массив предыдущих вершин
    p = [None]*n

    # Обнуляем кратчайший путь из стартовой вершины в себя.
    d[start] = 0

    # Вершины, расстояние до которых уже вычислено
    m0 = []
    # Вершины, расстояние до которых вычисляется
    m1 = [start]
    # Вершины, расстояние до которых ещё не вычислено
    m2 = [int(x) for x in range(n)]
    m2.remove(start)

    # Массив ребёр
    e = graph_list(m)

    while len(m1) != 0:
        v = m1.pop(0)
        m0.append(v)
        
        for edge in e:
            if edge[0] == v:

                # Если конец ребра принадлежит M2, 
                # то переносим его во множество M1
                # в конец очереди.
                # Обновляем расстояние до этой вершины.
                if edge[1] in m2:
                    m1.append(edge[1])
                    m2.remove(edge[1])
                    d[edge[1]] = d[v] + edge[-1]
                    p[edge[1]] = v

                # Если конец ребра принадлежит M1, 
                # то пытаемся уменьшить расстояние до этой вершины.
                if edge[1] in m1:
                    f = d[v] + edge[-1]
                    if d[edge[1]] > f:
                        d[edge[1]] = f
                        p[edge[1]] = v

                # Если конец ребра принадлежит M0,
                # и если DT можно уменьшить расстояние до этой вершины,
                # то обновляем расстояние, а конец ребра 
                # возвращаем в множество M1, помещая его в начало очереди.
                if edge[1] in m0 and d[edge[1]] > d[v] + edge[-1]:
                    d[edge[1]] = d[v] + edge[-1]
                    p[edge[1]] = v
                    m1.insert(0, edge[1])
    
    # Поиск кратчайшего пути
    route = [end]
    while end != start:
        end = p[route[-1]]
        route.append(end)
    return [d, route]


result = levit(matrix, start, end)
print("length =", result[0][end], "\npath =", result[1])
vis(matrix, result[1], "levit")

length = 7 
path = [7, 6, 5, 4, 0]
levit.html


##### Алгоритм Йена

In [20]:
def yen(m, start, end, k=1):
    
    # Количество вершин
    n = len(m)
    # Массив k кратчайших путей до заданной вершины
    paths = []
    # Массив k расстояний до заданной вершины
    lengths = []

    # Используем алгоритм Дейкстры для 
    # нахождение первого кратчайшего пути.
    result = dijkstra(m, start, end)
    paths.append(result[1][::-1])
    lengths.append(result[0][end])

    # Массив кандидатов кратчайших путей до заданной вершины
    canditates = []
    # Массив кандидатов расстояний до заданной вершины
    canditates_length = []

    # Ищем k кратчайших путей
    for h in range(k-1):
        for i in range(1, len(paths[h])):
            medit = copy.deepcopy(m)
            # Корневой путь, относительно которого будет ветвление
            root_path = paths[h][0:i]
            # Вершина, относительно которой будет ветвление
            spur_node = root_path[-1]
            # Длина корневого пути
            pre_lenght = 0

            for j in range(len(root_path)-1):
                pre_lenght += medit[root_path[j]][root_path[j+1]]
            
            # Удаляем вершины корневого пути из графа 
            # для удачного ветвления.
            for j in root_path:
                if j != spur_node:
                    medit[j] = [math.inf]*n

            # Удаляем все последние рёбра из предыдущих кратчайших путей,
            # когда ветвление начинается с начальной вершины, для удачного ветвления.
            if len(root_path) == 1:
                for forest in paths:
                    medit[forest[-2]][forest[-1]] = math.inf

            # Удаляем ребро из предыдущего пути
            # для удачного ветвления.
            medit[spur_node][paths[h][i]] = math.inf

            # Используем алгоритм Дейкстры для 
            # нахождение следующего кратчайшего пути.
            result = dijkstra(medit, spur_node, end)

            # Если такой путь существует, то полный путь его длину добавляем в кандидаты.
            if result[0][end] != math.inf:
                root_path.extend(result[1][-2::-1])
                pre_lenght += result[0][end]
                if root_path not in canditates and root_path not in paths:
                    canditates.append(root_path)
                    canditates_length.append(pre_lenght)

        # Среди кандидатов выбираем кратчайший путь.
        if len(canditates_length) != 0:
            best = canditates_length.index(min(canditates_length))
            if canditates[best] not in paths:
                paths.append(canditates.pop(best))
                lengths.append(canditates_length.pop(best))

        else: break

    return [lengths, paths]


result = yen(matrix, start, end, k=10)
for i in range(len(result[0])):
    print("length =", result[0][i], "\tpath =", result[1][i][::-1])
    vis(matrix, result[1][i][::-1], f"yen_{i}")

#print(result)

length = 7 	path = [7, 6, 5, 4, 0]
yen_0.html
length = 9 	path = [7, 2, 1, 0]
yen_1.html
length = 10 	path = [7, 0]
yen_2.html
length = 11 	path = [7, 3, 0]
yen_3.html


In [21]:
"""matrix = [[0, 2, 6, 8, math.inf, math.inf, 3, math.inf, math.inf],
          [2, 0, 9, 3, math.inf, 4, 9, math.inf, math.inf],
          [6, 9, 0, 7, math.inf, math.inf, math.inf, math.inf, math.inf],
          [8, 3, 7, 0, 5, 5, math.inf, math.inf, math.inf],
          [math.inf, math.inf, math.inf, 5, 0, math.inf, 8, 9, math.inf],
          [math.inf, 4, math.inf, 5, math.inf, 0, math.inf, 6, 4],
          [3, 9, math.inf, math.inf, 8, math.inf, 0, math.inf, math.inf],
          [math.inf, math.inf, math.inf, math.inf, 9, 6, math.inf, 0, 1],
          [math.inf, math.inf, math.inf, math.inf, math.inf, 4, math.inf, 1, 0]]"""

'matrix = [[0, 2, 6, 8, math.inf, math.inf, 3, math.inf, math.inf],\n          [2, 0, 9, 3, math.inf, 4, 9, math.inf, math.inf],\n          [6, 9, 0, 7, math.inf, math.inf, math.inf, math.inf, math.inf],\n          [8, 3, 7, 0, 5, 5, math.inf, math.inf, math.inf],\n          [math.inf, math.inf, math.inf, 5, 0, math.inf, 8, 9, math.inf],\n          [math.inf, 4, math.inf, 5, math.inf, 0, math.inf, 6, 4],\n          [3, 9, math.inf, math.inf, 8, math.inf, 0, math.inf, math.inf],\n          [math.inf, math.inf, math.inf, math.inf, 9, 6, math.inf, 0, 1],\n          [math.inf, math.inf, math.inf, math.inf, math.inf, 4, math.inf, 1, 0]]'

#### Тест скорости работы алгоритмов

In [22]:
def make_test(algorithm, m, start=0, end=4):
    ma = copy.deepcopy(m)
    
    count = time.time()
    algorithm(ma, start, end)
    result = (time.time() - count) * 1000

    return result

In [30]:
results_floyd_warshall = []
results_dijkstra = []
results_bellman_ford = []
results_johnson = []
results_levit = []
results_yen_first = []

graphs = read_adjancies('graphs.txt')[:-1]


for i in graphs:
    results_floyd_warshall.append(make_test(floyd_warshall, i))
    results_dijkstra.append(make_test(dijkstra, i))
    results_bellman_ford.append(make_test(bellman_ford, i))
    results_johnson.append(make_test(johnson, i))
    results_levit.append(make_test(levit, i))
    results_yen_first.append(make_test(yen, i))

results_floyd_warshall.append("|V|**3")
results_dijkstra.append("|V|**2 + m")
results_bellman_ford.append("|V|*|E|")
results_johnson.append("(|V|**2 + m)*|V|*|E|")
results_levit.append("|V|*|E|")
results_yen_first.append("(|V|**2 + m)*|V|*k")

data = {
    "nodes": [5, 7, 12, 20, 25, 30, "complexity"],
    "floyd_warshall": results_floyd_warshall,
    "dijkstra": results_dijkstra,
    "bellman_ford": results_bellman_ford,
    "johnson": results_johnson,
    "levit": results_levit,
    "yen": results_yen_first
}
#print(len(data["nodes"]), len(data["floyd_warshall"]), len(data["dijkstra"]), len(data["bellman_ford"]), len(data["johnson"]), len(data["levit"]), len(data["yen"]))

df = pd.DataFrame(data)
print(df)

        nodes floyd_warshall    dijkstra bellman_ford               johnson  \
0           5            0.0         0.0          0.0                   0.0   
1           7            0.0         0.0          0.0               0.99206   
2          12            0.0         0.0          0.0              0.991583   
3          20       4.006147    1.004934     3.043652              6.976843   
4          25       9.011984         0.0          0.0             11.997223   
5          30       8.994818    1.003265     2.968311             20.992517   
6  complexity         |V|**3  |V|**2 + m      |V|*|E|  (|V|**2 + m)*|V|*|E|   

      levit                 yen  
0       0.0                 0.0  
1       0.0                 0.0  
2       0.0                 0.0  
3       0.0            1.000881  
4  1.002312                 0.0  
5       0.0                 0.0  
6   |V|*|E|  (|V|**2 + m)*|V|*k  


### Вывод